In [1]:
import pandas as pd
import os
import numpy as np
import scipy as ss
import scipy.stats
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import print_function
import cobra
from corda import reaction_confidence
from corda import CORDA
data = "/Users/annasintsova/git_repos/"\
        "HUTI-RNAseq/data/get_homologs_output/C50_S90_e0_/"\
        "run_C50_S90_e0__pan_C50_S90/"\
        "2018-02-26_pangenome_matrix_t0_crossRef.csv"

In [2]:
def assignConfidence(x):
    if x > 1000.0:
        return 2 
    elif x < 100.0:
        return - 1
    else:
        return 0

In [3]:
def getModel(strain):
    model_dir = "/Users/annasintsova/git_repos/"\
        "HUTI-RNAseq/analysis/fba/2017-12-16-model-2/data"
    model_path = os.path.join(model_dir, "{}_homologues/{}_model/{}_iML1515.xml".format(strain, strain, strain)) 
    return cobra.io.read_sbml_model(model_path)

In [4]:
def binRPKMs(data, strain):
    """
    Take in counts, select rpkms for correct strain, assign them confidence, 
    return confidence for ur and uti conditions (2 dictionaries)
    """
    
    counts = pd.read_csv(data)
    rpkm = counts[["MG1655", "{}_UR_RPKM".format(strain), "{}_UTI_RPKM".format(strain)]]
    rpkm = rpkm.dropna(subset=["MG1655"])
    rpkm = rpkm[rpkm.MG1655 != "PARALOGS"]
    rpkm.set_index('MG1655', inplace=True)
    rpkm = rpkm.dropna()
    confidences = {'UR_conf':list(map(assignConfidence, rpkm["{}_UR_RPKM".format(strain)].values)),
               'UTI_conf':list(map(assignConfidence, rpkm["{}_UTI_RPKM".format(strain)].values))}
    confidence_df = pd.DataFrame(confidences,index = rpkm.index)
    ur_conf = confidence_df.UR_conf.to_dict()
    uti_conf = confidence_df.UTI_conf.to_dict()
    return ur_conf, uti_conf


In [9]:
x, y = binRPKMs(data, "HM43")
y

{'b0002': -1,
 'b0003': -1,
 'b0004': -1,
 'b0006': -1,
 'b0007': -1,
 'b0008': 0,
 'b0009': -1,
 'b0010': -1,
 'b0011': -1,
 'b0013': -1,
 'b0014': 0,
 'b0015': -1,
 'b0019': -1,
 'b0020': -1,
 'b0023': 0,
 'b0025': -1,
 'b0026': -1,
 'b0027': -1,
 'b0028': -1,
 'b0029': -1,
 'b0030': -1,
 'b0031': -1,
 'b0032': -1,
 'b0033': -1,
 'b0034': -1,
 'b0035': -1,
 'b0036': -1,
 'b0037': -1,
 'b0038': -1,
 'b0039': -1,
 'b0040': -1,
 'b0041': -1,
 'b0042': -1,
 'b0045': -1,
 'b0046': -1,
 'b0047': -1,
 'b0048': -1,
 'b0049': -1,
 'b0050': -1,
 'b0051': -1,
 'b0052': -1,
 'b0053': -1,
 'b0054': -1,
 'b0055': -1,
 'b0058': -1,
 'b0059': -1,
 'b0060': -1,
 'b0061': -1,
 'b0062': -1,
 'b0063': -1,
 'b0064': -1,
 'b0065': -1,
 'b0066': -1,
 'b0067': -1,
 'b0068': -1,
 'b0069': -1,
 'b0071': -1,
 'b0072': -1,
 'b0073': -1,
 'b0074': -1,
 'b0076': -1,
 'b0077': -1,
 'b0078': -1,
 'b0080': -1,
 'b0081': -1,
 'b0082': -1,
 'b0083': -1,
 'b0084': -1,
 'b0085': -1,
 'b0086': -1,
 'b0087': -1,
 'b0088':

In [10]:
def runCORDA(model, conf_dict, file_out, metabolite = ""):
    """
    
    """
    reaction_conf = {}
    for rx in model.reactions:
        if rx.id == "BIOMASS_Ec_iML1515_core_75p37M":
            reaction_conf[rx.id] = 3
        else:
            reaction_conf[rx.id] = reaction_confidence(rx.gene_reaction_rule, conf_dict)
    if not metabolite:
        opt = CORDA(model, reaction_conf)
        opt.build()
    else:
        opt = CORDA(model, reaction_conf, met_prod=metabolite)
        opt.build()
    print(opt)
   
    with open((file_out+".tab"), "w") as fh:
        for k, used in opt.included.items():
            if used:
                fh.write(opt.model.reactions.get_by_id(k).id +
                         "\t" + opt.model.reactions.get_by_id(k).name + "\t" +
                         opt.model.reactions.get_by_id(k).gene_reaction_rule + "\n")

    return opt.included

In [ ]:
z = runCORDA(getModel("HM43"), y, "../test")

In [38]:
def buildEnvModel(included_dict, model, strain, condition, output_dir):
    today = dt.datetime.today().strftime("%Y-%m-%d")
    print(len(model.reactions))

    all_reactions = list(model.reactions)
    for rx in all_reactions:
        if not included_dict[rx.id] :
            model.reactions.get_by_id(rx.id).remove_from_model()
        
    print(len(model.reactions))
    name_suffix = model.id.split("_")[-1]
    model.id = "{}_{}_{}".format(strain, condition, name_suffix)
    cobra.io.write_sbml_model(model, os.path.join(output_dir, (today + strain + condition +".sbml")))
    return model

In [45]:
def cordaAnalysis(strains, data, output_dir):
    today = dt.datetime.today().strftime("%Y-%m-%d")
    all_envs ={}
    all_models = []
    for strain in strains:
        print(strain)
        # get model
        print("Getting the model")
        model = getModel(strain)
        # bin
        print("Binning data")
        UR, UTI = binRPKMs(data, strain)
        # run CORDA
        # for urine
        print("Running CORDA for urine")
        UR_key = "{}_UR".format(strain)
        urine_env = runCORDA(model, UR, os.path.join(output_dir, today+UR_key+".tab"))   
        print("Running CORDA for UTI")
        UTI_key = "{}_UTI".format(strain)
        uti_env = runCORDA(model, UTI, os.path.join(output_dir, today+UTI_key+".tab"))
        all_envs[UR_key] = urine_env
        all_envs[UTI_key] = uti_env
    for key, val in all_envs.items():
        st = key.split("_")[0]
        condition = key.split("_")[1]
        print("Building Model")
        m = buildEnvModel(val, getModel(st), st, condition, output_dir)
        all_models.append(m)
        print("done")
    return all_envs, all_models

In [ ]:
# HM56, HM14, HM43, HM54, HM86
strains = ["HM86"]
output_dir = "../data/2018-03-27-CORDA-analysis"
cordaAnalysis(strains, data, output_dir)


HM86
Getting the model
Binning data
Running CORDA for urine
build status: reconstruction complete
Inc. reactions: 486/2607
 - unclear: 323/1326
 - exclude: 92/907
 - low and medium: 70/373
 - high: 1/1

Running CORDA for UTI
